## Transform

In [1]:
# Imports
import os
import json
import pandas as pd

In [2]:
# Reading extracted csvs
with open('collected_tweets_2021-03-15-21-26-25.txt', 'r') as file:
    tweets = file.readlines()

In [3]:
# Function to export json
def dump_my_json(json_structure):
    with open('tweet2.json', 'w') as f:
        json.dump(
            json_structure, 
            f
        )

In [4]:
# Fazendo parse dos tweets para um formato json
parsed_tweets = [json.loads(json.loads(i)) for i in tweets]

In [5]:
# Function to convert json to dataframe
def tweet_to_df(tweet):
    try:
        df_out = pd.DataFrame(tweet).reset_index(drop=True).iloc[:1]
        
        # Filter out retweets
        if df_out['text'][0].startswith('RT @'):
            return None
        
        # Bring up user information to the first level of our dataframe
        df_out['user_id'] = tweet['user']['id']
        df_out['user_screen_name'] = tweet['user']['screen_name']
        df_out['user_protected'] = tweet['user']['protected']
        df_out['user_verified'] = tweet['user']['verified']
        df_out['user_followers_count'] = tweet['user']['followers_count']
        df_out['user_friends_count'] = tweet['user']['friends_count']
        
        
        # Keep only relevant columns
        df_out = df_out[['id','text','user_id','user_screen_name','user_protected',
                         'user_verified','user_followers_count','user_friends_count']]
        
    except:
        return None
    return df_out


In [6]:
# Goes through fuction that transform data before stacking up
fixed_parsed_tweets = [tweet_to_df(tweet) for tweet in parsed_tweets]
fixed_parsed_tweets = [i for i in fixed_parsed_tweets if i is not None]

In [7]:
# Transform to pandas dataframe
df_tweets = pd.concat(fixed_parsed_tweets, ignore_index=True)

## Ingest data into local mySQL

In [8]:
# Imports
import pyodbc
from sqlalchemy import create_engine

In [9]:
# DB Connection Setup
mysql_credentials = {
    "hostname": 'localhost',
    "dbname": 'twitter',
    "uname": 'root',
    "pwd": 'root'
}

In [10]:
# Creates engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(
                        host=mysql_credentials["hostname"],
                        db=mysql_credentials["dbname"],
                        user=mysql_credentials["uname"],
                        pw=mysql_credentials["pwd"]))

In [11]:
# Ingest
df_tweets.to_sql("tweets", con=engine, index=False, if_exists="replace")